### Determining the optimal number of hidden layers and neurons for an Artificial Neural Network (ANN) 
This can be challenging and often requires experimentation. However, there are some guidelines and methods that can help you in making an informed decision:

- Start Simple: Begin with a simple architecture and gradually increase complexity if needed.
- Grid Search/Random Search: Use grid search or random search to try different architectures.
- Cross-Validation: Use cross-validation to evaluate the performance of different architectures.
- Heuristics and Rules of Thumb: Some heuristics and empirical rules can provide starting points, such as:
  -    The number of neurons in the hidden layer should be between the size of the input layer and the size of the output layer.
  -  A common practice is to start with 1-2 hidden layers.

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [10]:
data=pd.read_csv('Churn_Modelling.csv')
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

onehot_encoded_geo=OneHotEncoder(handle_unknown='ignore')
geo_encoded=onehot_encoded_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehot_encoded_geo.get_feature_names_out(['Geography']))

data=pd.concat([data.drop(['Geography'],axis=1),geo_encoded_df],axis=1)

x=data.drop(['Exited'],axis=1)
y=data['Exited']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Save The Pickle File
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)
## Save The Label Encoder
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)
## Save The One Hot Encoder
with open('onehot_encoded_geo.pkl','wb') as file:
    pickle.dump(onehot_encoded_geo,file)


In [11]:
## Define a function to create the model and try different parameters(KerasClassifier)

def create_model(neurons=32,layer=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layer-1):
        model.add(Dense(neurons,activation='relu'))



    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [29]:
## Create a Keras classifier
model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,verbose=1)

In [30]:
## Define the parameter grid
param_grid = {
    'neurons': [32, 64, 128],
    'layer': [1, 2, 3],
    'batch_size': [16, 32],
    'epochs': [50, 100]
}

In [31]:

# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, verbose=1)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

AttributeError: 'super' object has no attribute '__sklearn_tags__'